# 03. 단어 단위 RNN - 임베딩 사용
이번 챕터에서는 문자 단위가 아니라 RNN의 입력 단위를 단어 단위로 사용합니다. 그리고 단어 단위를 사용함에 따라서 Pytorch에서 제공하는 임베딩 층(embedding layer)를 사용하겠습니다.

## 1. 훈련 데이터 전처리하기
우선 실습을 위해 필요한 도구들을 임포트합니다.



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()


우리가 만들 RNN은 'Repeat is the best medicine for'을 입력받으면 'is the best medicine for memory'를 출력하는 RNN입니다. 위의 임의의 문장으로부터 단어장(vocabulary)을 만듭니다.



In [3]:
vocab = list(set(sentence))
print(vocab)

['for', 'memory', 'the', 'is', 'Repeat', 'best', 'medicine']


이제 단어장의 단어에 고유한 정수 인덱스를 부여합니다. 그리고 그와 동시에 모르는 단어를 의미하는 UNK 토큰도 추가하겠습니다.



In [5]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)} # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [6]:
print(word2index)

{'for': 1, 'memory': 2, 'the': 3, 'is': 4, 'Repeat': 5, 'best': 6, 'medicine': 7, '<unk>': 0}


In [7]:
print(word2index['memory'])

2


단어 'memory'와 맵핑되는 정수는 2입니다. 예측 단계에서 예측한 문장을 확인하기 위해 idx2word도 만듭니다.



In [8]:
index2word = {v:k for k, v in word2index.items()}
print(index2word)

{1: 'for', 2: 'memory', 3: 'the', 4: 'is', 5: 'Repeat', 6: 'best', 7: 'medicine', 0: '<unk>'}


In [9]:
print(index2word[2])

memory


정수 2와 맵핑되는 단어는 memory인 것을 확인할 수 있습니다. 이제 데이터의 각 단어를 정수로 인코딩하는 동시에, 입력 데이터와 레이블 데이터를 만드는 build_data라는 함수를 만들어보겠습니다.



In [10]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq
    
    

In [11]:
X, Y = build_data(sentence, word2index)

In [12]:
print(X) # Repeat is the best medicine for을 의미
print(Y) # is the best medicine for memory을 의미

tensor([[5, 4, 3, 6, 7, 1]])
tensor([[4, 3, 6, 7, 1, 2]])


***
## 2. 모델 구현하기
이제 모델을 설계합니다. 이전 모델들과 달라진 점은 임베딩 층을 추가했다는 점입니다. 파이토치에서는 nn.Embedding()을 사용해서 임베딩 층을 구현합니다. 임베딩층은 크게 두 가지 인자를 받는데 첫번째 인자는 단어장의 크기이며, 두번째 인자는 임베딩 벡터의 차원입니다.

In [52]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        
        # 워드 임베딩
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size,
                                           embedding_dim = input_size)
        
        # 입력 차원, 은닉 상태의 크기 정의
        self.rnn_layer = nn.LSTM(input_size, hidden_size, batch_first=batch_first)
        
        # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.
        self.linear = nn.Linear(hidden_size, vocab_size) 
        
         
    
    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)

        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)

        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)

        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)

        return output.view(-1, output.size(2))


이제 모델을 위해 하이퍼파라미터를 설정합니다.



In [53]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [54]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)

# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.

# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [55]:
print(X)
X.shape

tensor([[5, 4, 3, 6, 7, 1]])


torch.Size([1, 6])

In [56]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[-0.0163,  0.2441, -0.1079,  0.1568,  0.0312, -0.1119,  0.2071, -0.1297],
        [-0.0240,  0.2154, -0.0849,  0.1355,  0.0575, -0.1492,  0.2268, -0.0643],
        [-0.0360,  0.1431, -0.0038,  0.1098,  0.0949, -0.1680,  0.1739, -0.0428],
        [-0.0097,  0.1917, -0.0582,  0.1110,  0.0858, -0.1678,  0.1037, -0.1122],
        [-0.0329,  0.1267, -0.0311,  0.1188,  0.0723, -0.1837,  0.1475, -0.1362],
        [-0.0141,  0.2936, -0.1357,  0.1285,  0.0615, -0.1258,  0.1466, -0.0766]],
       grad_fn=<ViewBackward>)


모델이 어떤 예측값을 내놓기는 하지만 현재 가중치는 랜덤 초기화되어 있어 의미있는 예측값은 아닙니다. 예측값의 크기를 확인해봅시다.



In [57]:
print(output.shape)


torch.Size([6, 8])


예측값의 크기는 (6, 8)입니다. 이는 각각 (시퀀스의 길이, 은닉층의 크기)에 해당됩니다. 모델은 훈련시키기 전에 예측을 제대로 하고 있는지 예측된 정수 시퀀스를 다시 단어 시퀀스로 바꾸는 decode 함수를 만듭니다.



In [58]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

### 학습하기

In [59]:
for step in range(201):
    optimizer.zero_grad()
    
    output = model(X)
    
    loss = loss_function(output, Y.view(-1))
    
    loss.backward()
    
    optimizer.step()
    
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()


[01/201] 2.0793 
Repeat for best best for best for

[41/201] 1.8997 
Repeat the best best for for for

[81/201] 1.4965 
Repeat the the best medicine for memory

[121/201] 1.0006 
Repeat the the best medicine for memory

[161/201] 0.6526 
Repeat the the best medicine for memory

[201/201] 0.4413 
Repeat is the best medicine for memory



In [60]:
output.softmax(-1).argmax(-1).tolist()

[4, 3, 6, 7, 1, 2]

In [61]:
decode(pred)

['is', 'the', 'best', 'medicine', 'for', 'memory']